In [ ]:
!pip install latex2sympy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 2.6 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.7.2-py3-none-any.whl size=140930 sha256=f1e7ec1a0a673b0eb2791b3d411e4b5b842c5963aec8b5cd14986e9827526687
  Stored in directory: /root/.cache/pip/wheels/24/09/d5/906109a0b239813cee4d939fa6ccee45f63a19e3f774405e34
Successfully built antlr4-python3-runtime
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
omegaconf 2.3.0 requires antlr4-python3-runtime==4.9.*, but you have antlr4-p

In [ ]:
import os
import glob
import math
import pandas as pd
import numpy as np
from IPython.display import display

def _to_bool(x):
    if isinstance(x, bool):
        return x
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return False
    sx = str(x).strip().lower()
    if sx in {"true","1","yes","y","t"}:  return True
    if sx in {"false","0","no","n","f"}:  return False
    try:
        return bool(int(sx))
    except Exception:
        return False

def summarize_results_per_csv(folder_path: str, pattern: str = "results_*.csv"):
    """
    Resume, para cada CSV de resultados en una carpeta, cuántos ejercicios:
      - totales
      - correctos (is_correct=True)
      - incorrectos (is_correct=False)
      - marcados como solubles (is_octave_resolvable=True)
      - con error de ejecución (execution_error no nulo)
      - con código Octave generado (octave_code no nulo)

    Devuelve un DataFrame con una fila por archivo y muestra la tabla por pantalla.
    """
    folder_path = os.path.abspath(folder_path)
    csv_paths = sorted(glob.glob(os.path.join(folder_path, pattern)))

    if not csv_paths:
        print(f"❌ No se encontraron CSVs con patrón '{pattern}' en: {folder_path}")
        return None

    rows = []
    for p in csv_paths:
        try:
            df = pd.read_csv(p)
        except Exception as e:
            print(f"⚠️ Error leyendo {p}: {e}")
            continue

        # Asegurar columnas mínimas esperadas
        for c in ["row_id", "is_correct", "is_octave_resolvable",
                  "octave_code", "execution_error"]:
            if c not in df.columns:
                df[c] = None

        # Normalizaciones
        df["is_correct"] = df["is_correct"].fillna(False).astype(bool)
        df["is_octave_resolvable"] = df["is_octave_resolvable"].apply(_to_bool)

        n_rows   = len(df)
        n_problems = df["row_id"].nunique() if "row_id" in df.columns else n_rows

        n_correct     = int(df["is_correct"].sum())
        n_incorrect   = n_rows - n_correct

        n_resolvable  = int(df["is_octave_resolvable"].sum())
        n_non_resolv  = n_rows - n_resolvable

        n_with_code   = int(df["octave_code"].notna().sum())
        n_exec_error  = int(df["execution_error"].notna().sum())

        # Porcentajes respecto al nº de filas del CSV
        def pct(part):
            return part / n_rows * 100 if n_rows > 0 else 0.0

        rows.append({
            "file_name": os.path.basename(p),
            "n_rows": n_rows,
            "n_unique_problems": n_problems,

            "n_correct": n_correct,
            "n_incorrect": n_incorrect,
            "correct_%": pct(n_correct),
            "incorrect_%": pct(n_incorrect),

            "n_resolvable_true": n_resolvable,
            "n_resolvable_false": n_non_resolv,
            "resolvable_%": pct(n_resolvable),

            "n_with_code": n_with_code,
            "with_code_%": pct(n_with_code),

            "n_exec_error": n_exec_error,
            "exec_error_%": pct(n_exec_error),
        })

    if not rows:
        print("❌ No se pudo procesar ningún CSV válido.")
        return None

    summary_df = pd.DataFrame(rows)
    # Ordenar por nº de filas (o por correctos, si prefieres)
    summary_df = summary_df.sort_values("file_name").reset_index(drop=True)

    print(f"📂 Carpeta analizada: {folder_path}")
    print("=== RESUMEN POR CSV (conteos y porcentajes de estado de los ejercicios) ===\n")
    total_items = summary_df["n_rows"].sum()
    print(f"Cantidad total de ítems (filas) en todos los datasets: {total_items}")
    with pd.option_context("display.max_rows", 200,
                           "display.max_columns", 50,
                           "display.float_format", lambda x: f"{x:0.2f}"):
        display(summary_df)

    return summary_df

|index|dataset|problems|n\_unique\_problems|n\_correct|n\_incorrect|correct\_%|incorrect\_%|n\_resolvable\_true|n\_resolvable\_false|resolvable\_%|n\_with\_code|with\_code\_%|n\_exec\_error|exec\_error\_%|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|gsm8k|8792|8792|7604|1188|86\.49|13\.51|8792|0|100\.0|8792|100\.0|81|0\.92|
|1|MATH|10000|10000|4061|5939|40\.61 (46\.77)|59\.39 (53\.23)|9723|277|97\.23|9698|96\.98|1232|12\.32|

In [ ]:
summary_per_csv = summarize_results_per_csv("/content/drive/MyDrive/tesis/experiments_full_datasets")

📂 Carpeta analizada: /content/drive/MyDrive/tesis/experiments_full_datasets
=== RESUMEN POR CSV (conteos y porcentajes de estado de los ejercicios) ===

Cantidad total de ítems (filas) en todos los datasets: 21292


,file_name,n_rows,n_unique_problems,n_correct,n_incorrect,correct_%,incorrect_%,n_resolvable_true,n_resolvable_false,resolvable_%,n_with_code,with_code_%,n_exec_error,exec_error_%
0,results_full_gsm8k_cot_reasoning_mathstral-7b.csv,8792,8792,7604,1188,86.49,13.51,8792,0,100.00,8792,100.00,81,0.92
1,results_full_math_cot_reasoning_mathstral-7b.csv,12500,12500,5082,7418,40.66,59.34,12223,277,97.78,12198,97.58,1507,12.06


In [ ]:
summary_per_csv = summarize_results_per_csv("/content/drive/MyDrive/tesis/experiments_full_datasets", pattern = "corrected_results_*.csv")


📂 Carpeta analizada: /content/drive/MyDrive/tesis/experiments_full_datasets
=== RESUMEN POR CSV (conteos y porcentajes de estado de los ejercicios) ===

Cantidad total de ítems (filas) en todos los datasets: 21292


,file_name,n_rows,n_unique_problems,n_correct,n_incorrect,correct_%,incorrect_%,n_resolvable_true,n_resolvable_false,resolvable_%,n_with_code,with_code_%,n_exec_error,exec_error_%
0,corrected_results_full_gsm8k_cot_reasoning_mat...,8792,8792,7626,1166,86.74,13.26,8792,0,100.00,8792,100.00,46,0.52
1,corrected_results_full_math_cot_reasoning_math...,12500,12500,5176,7324,41.41,58.59,12223,277,97.78,12198,97.58,1115,8.92


In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pandas.api.types as ptypes

from latex2sympy2 import latex2sympy

def _strip_dollars(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    # elimina $ o $$ envolventes
    if s.startswith("$$") and s.endswith("$$"):
        return s[2:-2].strip()
    if s.startswith("$") and s.endswith("$"):
        return s[1:-1].strip()
    return s

def _eval_answer_latex(latex_str: str):
    """
    Intenta interpretar una expresión LaTeX como valor numérico usando latex2sympy2.
    Devuelve (valor_float, error_msg), donde error_msg es None si todo va bien.
    """
    if latex_str is None:
        return None, "Respuesta vacía"
    txt = _strip_dollars(latex_str)
    if not txt:
        return None, "Respuesta vacía"

    try:
        expr = latex2sympy(txt)
        # evalf() para casos como \pi, \sqrt{2}, \frac{1}{2}, etc.
        val = float(expr.evalf())
        return val, None
    except Exception as e:
        return None, f"Error interpretando LaTeX: {e}"

def edit_experiment_csv(csv_path: str):
    """
    Pequeño editor interactivo de CSV:
      - Botones anterior / siguiente registro
      - Textbox + botón para saltar a la fila N (dentro de las filas visibles)
      - Edición de campos (todas las columnas)
      - Filtro sencillo por expresión pandas.query (sobre el DataFrame)
      - Botón para guardar cambios en el CSV original
      - Cálculo automático de la expresión LaTeX en la columna 'answer' (si existe)
    """
    # =========================
    # 1) Cargar CSV
    # =========================
    df = pd.read_csv(csv_path)
    if df.empty:
        print(f"⚠️ El CSV '{csv_path}' está vacío.")
        return

    # Estado interno
    state = {
        "df": df,
        "original_dtypes": df.dtypes.to_dict(),
        "csv_path": csv_path,
        "filtered_idx": list(df.index),  # índices reales visibles
        "view_pos": 0,                   # posición dentro de filtered_idx (0-based)
        "filter_expr": ""
    }

    # =========================
    # 2) Widgets básicos
    # =========================
    lbl_file = widgets.HTML(f"<b>Archivo:</b> {csv_path}")

    # Navegación básica
    btn_prev = widgets.Button(description="⟨ Anterior", button_style="")
    btn_next = widgets.Button(description="Siguiente ⟩", button_style="")
    idx_display = widgets.IntText(
        value=1,
        description="Fila:",
        disabled=True,
        layout=widgets.Layout(width="140px")
    )
    total_display = widgets.Label()

    # Textbox + botón para saltar a una fila concreta
    jump_input = widgets.IntText(
        value=1,
        description="Ir a fila:",
        layout=widgets.Layout(width="160px")
    )
    btn_jump = widgets.Button(description="Ir", button_style="info")

    # Guardar (arriba)
    btn_save_row = widgets.Button(description="💾 Guardar fila", button_style="info")
    btn_save_csv = widgets.Button(description="💾 Guardar CSV", button_style="success")

    # Filtro
    txt_filter = widgets.Text(
        value="",
        description="Filtro (query):",
        layout=widgets.Layout(width="80%")
    )
    btn_apply_filter = widgets.Button(description="Aplicar filtro", button_style="primary")
    btn_clear_filter = widgets.Button(description="Limpiar filtro", button_style="warning")
    lbl_filter_status = widgets.HTML(value="<i>Sin filtro aplicado.</i>")

    # Zona de mensajes
    out_msg = widgets.Output()

    # =========================
    # 3) Widgets por columna (editor de fila)
    # =========================
    col_widgets = {}
    col_boxes = []
    for col in df.columns:
        dtype = state["original_dtypes"][col]
        # Booleanos como Checkbox, resto como Textarea
        if ptypes.is_bool_dtype(dtype) or col.lower().startswith("is_"):
            w = widgets.Checkbox(description=col, indent=False)
        else:
            w = widgets.Textarea(
                description=col,
                layout=widgets.Layout(width="100%", height="60px")
            )
        col_widgets[col] = w
        col_boxes.append(w)

    editor_box = widgets.VBox(col_boxes)

    # =========================
    # 3.1) Widget para valor numérico de 'answer'
    # =========================
    answer_value = widgets.Text(
        value="",
        description="answer =",
        disabled=True,
        layout=widgets.Layout(width="300px")
    )

    # =========================
    # 4) Funciones auxiliares
    # =========================
    def _current_real_index():
        """Índice real del DataFrame para la fila actualmente visible."""
        if not state["filtered_idx"]:
            return None
        pos = max(0, min(state["view_pos"], len(state["filtered_idx"]) - 1))
        return state["filtered_idx"][pos]

    def _update_total_label():
        total = len(state["filtered_idx"])
        idx_display.value = (state["view_pos"] + 1) if total > 0 else 0
        total_display.value = f" / {total} filas visibles (total en CSV: {len(state['df'])})"

    def _update_answer_value(row):
        """Calcula y muestra el valor numérico de la columna 'answer' (si existe)."""
        if "answer" not in state["df"].columns:
            answer_value.value = ""
            return
        latex_ans = row.get("answer", "")
        val, err = _eval_answer_latex(latex_ans)
        if err is not None:
            answer_value.value = f"[err] {err}"
        else:
            answer_value.value = f"{val:.15g}"

    def _load_row_into_widgets():
        """Carga la fila actual en los widgets de edición y evalúa 'answer'."""
        with out_msg:
            out_msg.clear_output()
        ridx = _current_real_index()
        if ridx is None:
            for col, w in col_widgets.items():
                if isinstance(w, widgets.Checkbox):
                    w.value = False
                else:
                    w.value = ""
            answer_value.value = ""
            with out_msg:
                print("⚠️ No hay filas que mostrar (filtro dejó vista vacía).")
            _update_total_label()
            return

        row = state["df"].loc[ridx]
        for col, w in col_widgets.items():
            val = row.get(col, "")
            if isinstance(w, widgets.Checkbox):
                w.value = bool(val) if not pd.isna(val) else False
            else:
                w.value = "" if pd.isna(val) else str(val)

        # Actualizar valor numérico de 'answer'
        _update_answer_value(row)
        _update_total_label()

    def _cast_value(col, text_value):
        """Intenta castear el valor introducido al tipo original de la columna."""
        orig_dtype = state["original_dtypes"].get(col, object)

        if ptypes.is_bool_dtype(orig_dtype):
            s = str(text_value).strip().lower()
            if s in {"true", "1", "yes", "y", "t"}:
                return True
            if s in {"false", "0", "no", "n", "f"}:
                return False
            return False

        if ptypes.is_integer_dtype(orig_dtype):
            try:
                return int(text_value)
            except Exception:
                return np.nan

        if ptypes.is_float_dtype(orig_dtype):
            try:
                return float(text_value)
            except Exception:
                return np.nan

        # Por defecto, string
        return text_value

    def _save_widgets_into_row():
        """Guarda en el DataFrame la fila actual editada."""
        ridx = _current_real_index()
        if ridx is None:
            with out_msg:
                out_msg.clear_output()
                print("⚠️ No hay fila actual para guardar.")
            return

        for col, w in col_widgets.items():
            if isinstance(w, widgets.Checkbox):
                val = bool(w.value)
            else:
                val = _cast_value(col, w.value)
            state["df"].at[ridx, col] = val

        with out_msg:
            out_msg.clear_output()
            print(f"✅ Fila {ridx} actualizada en memoria (aún no guardada en CSV).")

        # Recalcular valor de answer por si lo editaste
        row = state["df"].loc[ridx]
        _update_answer_value(row)

    def _apply_filter(expr: str):
        """Aplica un filtro usando pandas.query sobre el DataFrame."""
        expr = expr.strip()
        if not expr:
            state["filtered_idx"] = list(state["df"].index)
            state["view_pos"] = 0
            state["filter_expr"] = ""
            lbl_filter_status.value = "<i>Sin filtro aplicado.</i>"
            _load_row_into_widgets()
            return

        try:
            filtered = state["df"].query(expr, engine="python")
            state["filtered_idx"] = list(filtered.index)
            state["view_pos"] = 0
            state["filter_expr"] = expr

            if len(filtered) == 0:
                lbl_filter_status.value = (
                    f"<span style='color:orange'>Filtro aplicado, 0 filas coinciden: "
                    f"<code>{expr}</code></span>"
                )
            else:
                lbl_filter_status.value = (
                    f"<span style='color:green'>Filtro aplicado: "
                    f"<code>{expr}</code> → {len(filtered)} filas</span>"
                )
            _load_row_into_widgets()
        except Exception as e:
            lbl_filter_status.value = (
                f"<span style='color:red'>Error en filtro: {e}</span>"
            )

    # =========================
    # 5) Callbacks de botones
    # =========================
    def on_prev_clicked(_):
        if not state["filtered_idx"]:
            return
        state["view_pos"] = max(0, state["view_pos"] - 1)
        _load_row_into_widgets()

    def on_next_clicked(_):
        if not state["filtered_idx"]:
            return
        state["view_pos"] = min(len(state["filtered_idx"]) - 1, state["view_pos"] + 1)
        _load_row_into_widgets()

    def on_jump_clicked(_):
        """Ir a la fila N (1-based) dentro de las filas visibles."""
        with out_msg:
            out_msg.clear_output()

        if not state["filtered_idx"]:
            with out_msg:
                print("⚠️ No hay filas visibles para saltar (revisa el filtro).")
            return

        n = jump_input.value  # fila en numeración humana (1..N)
        total = len(state["filtered_idx"])
        if n < 1 or n > total:
            with out_msg:
                print(f"⚠️ Fila fuera de rango. Debe estar entre 1 y {total}.")
            return

        # Convertir a índice interno (0-based) y cargar
        state["view_pos"] = n - 1
        _load_row_into_widgets()

        with out_msg:
            print(f"➡ Saltado a la fila visible #{n}.")

    def on_save_row_clicked(_):
        _save_widgets_into_row()

    def on_save_csv_clicked(_):
        state["df"].to_csv(state["csv_path"], index=False)
        with out_msg:
            out_msg.clear_output()
            print(f"💾 CSV guardado en: {state['csv_path']}")

    def on_apply_filter_clicked(_):
        _apply_filter(txt_filter.value)

    def on_clear_filter_clicked(_):
        txt_filter.value = ""
        _apply_filter("")

    btn_prev.on_click(on_prev_clicked)
    btn_next.on_click(on_next_clicked)
    btn_jump.on_click(on_jump_clicked)
    btn_save_row.on_click(on_save_row_clicked)
    btn_save_csv.on_click(on_save_csv_clicked)
    btn_apply_filter.on_click(on_apply_filter_clicked)
    btn_clear_filter.on_click(on_clear_filter_clicked)

    # =========================
    # 6) Layout final y primera carga
    # =========================
    nav_box = widgets.HBox([
        btn_prev,
        btn_next,
        idx_display,
        total_display,
        jump_input,
        btn_jump,
    ])
    save_box = widgets.HBox([btn_save_row, btn_save_csv])
    filter_box = widgets.HBox([txt_filter, btn_apply_filter, btn_clear_filter])

    # Panel para mostrar el valor numérico de 'answer'
    answer_box = widgets.HBox([answer_value])

    ui = widgets.VBox([
        lbl_file,
        widgets.HTML("<hr>"),
        save_box,
        widgets.HTML("<hr>"),
        filter_box,
        lbl_filter_status,
        widgets.HTML("<hr>"),
        nav_box,
        answer_box,
        editor_box,
        widgets.HTML("<hr>"),
        widgets.HTML("<b>Mensajes:</b>"),
        out_msg
    ])

    display(ui)
    _load_row_into_widgets()

is_correct == False & execution_output

In [ ]:
edit_experiment_csv("/content/drive/MyDrive/tesis/experiments_full_datasets/corrected_results_full_math_cot_reasoning_mathstral-7b.csv")

In [ ]:
import json
import os

def count_json_items(json_path: str):
    """
    Cuenta la cantidad de ítems en un dataset JSON de forma robusta.

    - Si el archivo es un JSON normal:
        * lista  -> número de elementos
        * dict   -> número de claves
    - Si es JSONL (un objeto JSON por línea):
        * cuenta cuántas líneas son JSON válidos
    - Si nada de lo anterior funciona:
        * cuenta líneas no vacías como aproximación
    """
    if not os.path.isfile(json_path):
        print(f"⚠️ Archivo no encontrado: {json_path}")
        return None

    # 1) Intentar como JSON "normal" (lista o dict)
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        if isinstance(data, list):
            n = len(data)
            print(f"📏 El archivo es un JSON con una LISTA de {n} ítems.")
            return n

        if isinstance(data, dict):
            n = len(data)
            print(f"📁 El archivo es un JSON con un DICCIONARIO de {n} claves (ítems).")
            return n

        print(f"⚠️ El JSON raíz es de tipo {type(data).__name__}, no lista ni dict.")
        return None

    except json.JSONDecodeError as e:
        print(f"⚠️ No es un JSON único. Intentando como JSONL (uno por línea)... ({e})")

    # 2) Intentar como JSONL: un objeto JSON por línea
    jsonl_count = 0
    jsonl_errors = 0
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    _ = json.loads(line)
                    jsonl_count += 1
                except json.JSONDecodeError:
                    jsonl_errors += 1

        if jsonl_count > 0 and jsonl_errors == 0:
            print(f"📏 El archivo parece JSONL: {jsonl_count} objetos JSON (uno por línea).")
            return jsonl_count

        if jsonl_count > 0 and jsonl_errors > 0:
            print(f"📏 El archivo parece JSONL parcial: {jsonl_count} líneas válidas, {jsonl_errors} inválidas.")
            return jsonl_count

    except Exception as e:
        print(f"⚠️ Error leyendo como JSONL: {e}")

    # 3) Último recurso: contar líneas no vacías
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            non_empty = [ln for ln in f if ln.strip()]
        n = len(non_empty)
        print(f"ℹ️ No se pudo parsear como JSON/JSONL, pero el archivo tiene {n} líneas no vacías.")
        return n
    except Exception as e:
        print(f"⚠️ Error final leyendo el archivo: {e}")
        return None

In [ ]:
count_json_items("/content/drive/MyDrive/tesis/datasets/GSM-8K/gsm8k_completo.jsonl")

⚠️ No es un JSON único. Intentando como JSONL (uno por línea)... (Extra data: line 2 column 1 (char 313))
📏 El archivo parece JSONL: 8792 objetos JSON (uno por línea).


8792

In [ ]:
import json
from datasets import load_dataset, Dataset, concatenate_datasets

# 1. Cargar los conjuntos de datos
print("Cargando conjuntos de datos (entrenamiento y prueba)...")
train_dataset = load_dataset("openai/gsm8k", "main", split="train")
test_dataset = load_dataset("openai/gsm8k", "main", split="test")

# 2. Combinar ambos conjuntos en uno solo
# La función concatenate_datasets de Hugging Face es la forma más eficiente de hacerlo.
print("Combinando los datasets...")
full_dataset = concatenate_datasets([train_dataset, test_dataset])

print(f"Dataset combinado con éxito. Total de ejemplos: {len(full_dataset)}")

# 3. Guardar el dataset combinado en un solo archivo .jsonl
output_filename = "/content/drive/MyDrive/tesis/datasets/GSM-8K/gsm8k_completo.jsonl"

print(f"Guardando en '{output_filename}'...")

# El método .to_json() de Hugging Face es la forma más sencilla de exportarlo.
# El argumento 'lines=True' asegura el formato .jsonl (JSON Lines).
full_dataset.to_json(output_filename, orient="records", lines=True)

print(f"Dataset guardado completo en '{output_filename}'.")

Cargando conjuntos de datos (entrenamiento y prueba)...
Combinando los datasets...
Dataset combinado con éxito. Total de ejemplos: 8792
Guardando en '/content/drive/MyDrive/tesis/datasets/GSM-8K/gsm8k_completo.jsonl'...


Creating json from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Dataset guardado completo en '/content/drive/MyDrive/tesis/datasets/GSM-8K/gsm8k_completo.jsonl'.


In [ ]:
import json
import pandas as pd
from datasets import load_dataset

# El nombre del repositorio en Hugging Face
HF_DATASET_NAME = "qwedsacf/competition_math"
output_filename = "/content/drive/MyDrive/tesis/datasets/MATH/MATH_completo.jsonl"

print(f"Cargando dataset completo desde '{HF_DATASET_NAME}' (solo split 'train')...")

# Cargar únicamente el conjunto de entrenamiento, que contiene todos los datos disponibles
full_dataset = load_dataset(HF_DATASET_NAME, split="train")

print(f"Dataset cargado con éxito. Total de ejemplos: {len(full_dataset)}")

# Guardar el dataset completo en un solo archivo .jsonl
print(f"Guardando en formato .jsonl en '{output_filename}'...")

# El método .to_json() de Hugging Face es la forma más sencilla de exportar a JSON Lines.
full_dataset.to_json(
    output_filename,
    orient="records", # Formato de lista de objetos (registros)
    lines=True         # Clave para asegurar el formato JSON Lines (.jsonl)
)

print(f"Dataset guardado completo en '{output_filename}'.")

Cargando dataset completo desde 'qwedsacf/competition_math' (solo split 'train')...
Dataset cargado con éxito. Total de ejemplos: 12500
Guardando en formato .jsonl en '/content/drive/MyDrive/tesis/datasets/MATH/MATH_completo.jsonl'...


Creating json from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Dataset guardado completo en '/content/drive/MyDrive/tesis/datasets/MATH/MATH_completo.jsonl'.


In [ ]:
edit_experiment_csv("/content/drive/MyDrive/tesis/experiments_full_datasets/results_full_gsm8k_cot_reasoning_mathstral-7b.csv")